# --- Day 7: Some Assembly Required ---
This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates! Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.

Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from 0 to 65535). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

The included instructions booklet describes how to connect the parts together: x AND y -> z means to connect wires x and y to an AND gate, and then connect its output to wire z.

For example:

- 123 -> x means that the signal 123 is provided to wire x.
- x AND y -> z means that the bitwise AND of wire x and wire y is provided to wire z.
- p LSHIFT 2 -> q means that the value from wire p is left-shifted by 2 and then provided to wire q.
- NOT e -> f means that the bitwise complement of the value from wire e is provided to wire f.

Other possible gates include OR (bitwise OR) and RSHIFT (right-shift). If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.

In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to wire a?

In [204]:
def getWires():
    with open('wires.txt') as file:
        return file.read()
    
getWires().split('\n')

['NOT dq -> dr',
 'kg OR kf -> kh',
 'ep OR eo -> eq',
 '44430 -> b',
 'NOT gs -> gt',
 'dd OR do -> dp',
 'eg AND ei -> ej',
 'y AND ae -> ag',
 'jx AND jz -> ka',
 'lf RSHIFT 2 -> lg',
 'z AND aa -> ac',
 'dy AND ej -> el',
 'bj OR bi -> bk',
 'kk RSHIFT 3 -> km',
 'NOT cn -> co',
 'gn AND gp -> gq',
 'cq AND cs -> ct',
 'eo LSHIFT 15 -> es',
 'lg OR lm -> ln',
 'dy OR ej -> ek',
 'NOT di -> dj',
 '1 AND fi -> fj',
 'kf LSHIFT 15 -> kj',
 'NOT jy -> jz',
 'NOT ft -> fu',
 'fs AND fu -> fv',
 'NOT hr -> hs',
 'ck OR cl -> cm',
 'jp RSHIFT 5 -> js',
 'iv OR jb -> jc',
 'is OR it -> iu',
 'ld OR le -> lf',
 'NOT fc -> fd',
 'NOT dm -> dn',
 'bn OR by -> bz',
 'aj AND al -> am',
 'cd LSHIFT 15 -> ch',
 'jp AND ka -> kc',
 'ci OR ct -> cu',
 'gv AND gx -> gy',
 'de AND dk -> dm',
 'x RSHIFT 5 -> aa',
 'et RSHIFT 2 -> eu',
 'x RSHIFT 1 -> aq',
 'ia OR ig -> ih',
 'bk LSHIFT 1 -> ce',
 'y OR ae -> af',
 'NOT ca -> cb',
 'e AND f -> h',
 'ia AND ig -> ii',
 'ck AND cl -> cn',
 'NOT jh -> ji'

In [202]:
wires = getWires().split('\n')

#Formatting
wiresC = [] #Wires cleaned
for i in wires: #Loops through each instruction and 
    if 'NOT' in i: #Parses instruction if it's based on the 'NOT' operator
        #Puts instruction into form ['NOT', 'variable', 'NewVariable']
        wiresC.append(['NOT', i[4:i.index(' -> ')], i[i.index(' -> ')+4:]])
        
    elif 'AND' in i: #Parses instruction if it's based on the 'AND' operator
        #Puts instruction into form ['AND', 'variable1', 'variable2', 'NewVariable']
        try:
            wiresC.append(['AND', int(i[:i.index(' ')]), i[i.index('AND')+4:i.index(' -> ')], i[i.index(' -> ')+4:]])
        except ValueError:
            wiresC.append(['AND', i[:i.index(' ')], i[i.index('AND')+4:i.index(' -> ')], i[i.index(' -> ')+4:]])
            
    elif 'OR' in i: #Parses instruction if it's based on the 'OR' operator
        #Puts instruction into form ['AND', 'variable1', 'variable2', 'NewVariable']
        wiresC.append(['OR', i[:i.index(' ')], i[i.index('OR')+3:i.index(' -> ')], i[i.index(' -> ')+4:]])
    
    elif 'LSHIFT' in i: #Parses instruction if it's based on the 'LSHIFT' operator
        #Puts instruction into form ['LSHIFT', 'variable1', 'by how much', 'NewVariable']
        wiresC.append(['LSHIFT', i[:i.index(' ')], int(i[i.index('SHIFT')+6:i.index(' -> ')]), i[i.index('->')+3:]])
    
    elif 'RSHIFT' in i: #Parses instruction if it's based on the 'RSHIFT' operator
        #Puts instruction into form ['RSHIFT', 'variable1', 'by how much', 'NewVariable']
        wiresC.append(['RSHIFT', i[:i.index(' ')], int(i[i.index('SHIFT')+6:i.index(' -> ')]), i[i.index('->')+3:]])
    
    else: #Parses instruction if it's just assigning a value to a signal without an operator
        #Puts instruction into form ['signalNumber', 'NewVariable']
        try:
            wiresC.append([int(i[:i.index(' -> ')]), i[i.index(' -> ')+4:]])
        except ValueError:
            wiresC.append([i[:i.index(' -> ')], i[i.index(' -> ')+4:]])
            
def getSignals(wiresC):
    """
    This funciton takes in wiresC (wire instructions cleaned) and loops through every instruction.
    It takes each instruction and based on the operator (or lack thereof) will calculate it if all variables
    are available, otherwise it'll pass it.
    It'll loop through this process until all variables are filled.
    """
    wireValues = {} #Stores all wire signals
    while len(wireValues) != len(wiresC): #Loops through until all wires have a signal
        for i in wiresC: #Loops through instructions
            if i[0] == 'NOT': #Calculate NOT bitwise operation
                try:
                    wireValues.update({i[2]:int(''.join(['1' if x == '0' else '0' for x in bin(wireValues[i[1]])[2:]]), 2)})
                except KeyError:
                    continue
                    
            elif i[0] == 'AND': #Calculate AND bitwise operation
                if type(i[1]) == int:
                    try:
                        wireValues.update({i[3]:i[1] & wireValues[i[2]]})
                    except KeyError:
                        continue
                else:
                    try:
                        wireValues.update({i[3]:wireValues[i[1]] & wireValues[i[2]]})
                    except KeyError:
                        continue
                        
            elif i[0] == 'OR': #Calculate OR bitwise operation
                try:
                    wireValues.update({i[3]:wireValues[i[1]] | wireValues[i[2]]})
                except KeyError:
                    continue
                    
            elif i[0] == 'LSHIFT': #Calculate LEFT SHIFT bitwise operation
                try:
                    wireValues.update({i[3]:wireValues[i[1]] << i[2]})
                except KeyError:
                    continue
                    
            elif i[0] == 'RSHIFT': #Calculate RIGHT SHIFT bitwise operation
                try:
                    wireValues.update({i[3]:wireValues[i[1]] >> i[2]})
                except KeyError:
                    continue
                    
            else: #Assigns value to wire
                if type(i[0]) == int:
                    wireValues.update({i[1]:i[0]})
                else:
                    try:
                        wireValues.update({i[1]:wireValues[i[0]]})
                    except KeyError:
                        continue
                        
    return wireValues #Returns the dictionary of all wire signals
            
print(f"Signal for wire a: {getValues(wiresC)['a']}")

Signal for wire a: 3176


# --- Part Two ---
Now, take the signal you got on wire a, override wire b to that signal, and reset the other wires (including wire a). What new signal is ultimately provided to wire a?

In [201]:
wires = getWires().split('\n')

#Formatting
wiresC = [] #Wires cleaned
for i in wires: #Loops through each instruction and 
    if 'NOT' in i: #Parses instruction if it's based on the 'NOT' operator
        #Puts instruction into form ['NOT', 'variable', 'NewVariable']
        wiresC.append(['NOT', i[4:i.index(' -> ')], i[i.index(' -> ')+4:]])
        
    elif 'AND' in i: #Parses instruction if it's based on the 'AND' operator
        #Puts instruction into form ['AND', 'variable1', 'variable2', 'NewVariable']
        try:
            wiresC.append(['AND', int(i[:i.index(' ')]), i[i.index('AND')+4:i.index(' -> ')], i[i.index(' -> ')+4:]])
        except ValueError:
            wiresC.append(['AND', i[:i.index(' ')], i[i.index('AND')+4:i.index(' -> ')], i[i.index(' -> ')+4:]])
            
    elif 'OR' in i: #Parses instruction if it's based on the 'OR' operator
        #Puts instruction into form ['AND', 'variable1', 'variable2', 'NewVariable']
        wiresC.append(['OR', i[:i.index(' ')], i[i.index('OR')+3:i.index(' -> ')], i[i.index(' -> ')+4:]])
    
    elif 'LSHIFT' in i: #Parses instruction if it's based on the 'LSHIFT' operator
        #Puts instruction into form ['LSHIFT', 'variable1', 'by how much', 'NewVariable']
        wiresC.append(['LSHIFT', i[:i.index(' ')], int(i[i.index('SHIFT')+6:i.index(' -> ')]), i[i.index('->')+3:]])
    
    elif 'RSHIFT' in i: #Parses instruction if it's based on the 'RSHIFT' operator
        #Puts instruction into form ['RSHIFT', 'variable1', 'by how much', 'NewVariable']
        wiresC.append(['RSHIFT', i[:i.index(' ')], int(i[i.index('SHIFT')+6:i.index(' -> ')]), i[i.index('->')+3:]])
    
    else: #Parses instruction if it's just assigning a value to a signal without an operator
        #Puts instruction into form ['signalNumber', 'NewVariable']
        try:
            wiresC.append([int(i[:i.index(' -> ')]), i[i.index(' -> ')+4:]])
        except ValueError:
            wiresC.append([i[:i.index(' -> ')], i[i.index(' -> ')+4:]])
            
def getSignals(wiresC):
    """
    This funciton takes in wiresC (wire instructions cleaned) and loops through every instruction.
    It takes each instruction and based on the operator (or lack thereof) will calculate it if all variables
    are available, otherwise it'll pass it.
    It'll loop through this process until all variables are filled.
    """
    wireValues = {} #Stores all wire signals
    while len(wireValues) != len(wiresC): #Loops through until all wires have a signal
        for i in wiresC: #Loops through instructions
            if i[0] == 'NOT': #Calculate NOT bitwise operation
                try:
                    wireValues.update({i[2]:int(''.join(['1' if x == '0' else '0' for x in bin(wireValues[i[1]])[2:]]), 2)})
                except KeyError:
                    continue
                    
            elif i[0] == 'AND': #Calculate AND bitwise operation
                if type(i[1]) == int:
                    try:
                        wireValues.update({i[3]:i[1] & wireValues[i[2]]})
                    except KeyError:
                        continue
                else:
                    try:
                        wireValues.update({i[3]:wireValues[i[1]] & wireValues[i[2]]})
                    except KeyError:
                        continue
                        
            elif i[0] == 'OR': #Calculate OR bitwise operation
                try:
                    wireValues.update({i[3]:wireValues[i[1]] | wireValues[i[2]]})
                except KeyError:
                    continue
                    
            elif i[0] == 'LSHIFT': #Calculate LEFT SHIFT bitwise operation
                try:
                    wireValues.update({i[3]:wireValues[i[1]] << i[2]})
                except KeyError:
                    continue
                    
            elif i[0] == 'RSHIFT': #Calculate RIGHT SHIFT bitwise operation
                try:
                    wireValues.update({i[3]:wireValues[i[1]] >> i[2]})
                except KeyError:
                    continue
                    
            else: #Assigns value to wire
                if type(i[0]) == int:
                    wireValues.update({i[1]:i[0]})
                else:
                    try:
                        wireValues.update({i[1]:wireValues[i[0]]})
                    except KeyError:
                        continue
                        
    return wireValues #Returns the dictionary of all wire signals
            
for i in range(len(wiresC)): #Loops through instructions
    if wiresC[i][-1] == 'b': #If the instruction is assigning a value to 'b' change the value to the result of 'a'
        wiresC[i][0] = getSignals(wiresC)['a']

print(f"New signal for wire a: {getSignals(wiresC)['a']}")

New signal for wire a: 14710
